# KNN
## -> without removing stopwords, with stemming
- Accuracy: around 0.86 for k=20 (up to 0.87 for k = 50)
- F1-value:
- Roc-curve:

In [11]:
import pandas as pd
df = pd.read_csv('dataset.csv')
df = df.sample(30000)
df_target = df['humor']
df_data = df.copy()
df_data.drop(columns='humor')

df_target.head()

16744      True
116523    False
95357      True
89268      True
106192    False
Name: humor, dtype: bool

Preprocessing

In [12]:
from sklearn import preprocessing

#encode target to numeric
label_encoder = preprocessing.LabelEncoder()
df_target = label_encoder.fit_transform(df_target)
#df_target

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import re, string

#when running for the first time you need to activate this line for once.
#nltk.download('stopwords')

#definition of stemming function
token_pattern = re.compile(r"(?u)\b\w\w+\b") # split on whitespace

def tokenize(text):
    stemmer = PorterStemmer()
    stems = []
    
    tokens = token_pattern.findall(text)
    for item in tokens:
        stems.append(stemmer.stem(item))
    return stems

In [14]:
#I'm not sure if we agreed to use stemming or not. I'll store the results in another df
#Stopwords are removed here as well
stem_vectorizer = TfidfVectorizer(tokenizer=tokenize, min_df=0.01)
matrix = stem_vectorizer.fit_transform(df_data['text'])

df_data_stemmed = pd.DataFrame(matrix.toarray(), columns=stem_vectorizer.get_feature_names())
#display(df_data_stemmed)


C:\Users\mcl.NB-MCL\anaconda3\envs\datamining\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Create train/test split
df_data_train, df_data_test, df_target_train, df_target_test = train_test_split(
    df_data_stemmed, df_target, test_size=0.2, random_state=42)

#dt = DecisionTreeClassifier()

#for n_neighbors in range(20,25):#[1,2,3,4,5]:#1,2,3,4,5
#    knn_estimator = KNeighborsClassifier(n_neighbors)
#    knn_estimator.fit(df_data_train, df_target_train)
#    df_prediction = knn_estimator.predict(df_data_test)
#    print("k= {} acc: {}".format(n_neighbors, accuracy_score(df_target_test, df_prediction)))

grid_params = {
    'n_neighbors': [18,25,22],
    'algorithm': ['auto','ball_tree','kd_tree','brute'],
    'metric': ['euclidean','cosine']

}
knn_estimator = KNeighborsClassifier()

estimator = GridSearchCV(knn_estimator,grid_params,cv=3)
estimator.fit(df_data_train, df_target_train)
print(estimator.best_estimator_)
#print(estimator.best_score_)
#print(estimator.best_params_)
final_estimator_knn = estimator.best_estimator_

# final_estimator_knn.fit(df_data_train,df_target_train)

df_prediction = final_estimator_knn.predict(df_data_test)

print("k= {} acc: {}".format(estimator.best_params_, accuracy_score(df_target_test, df_prediction)))



#estimator.fit(df_data_train, df_target_train)
#df_prediction = knn_estimator.predict(df_data_test)
#print("k= {} acc: {}".format('', accuracy_score)


C:\Users\mcl.NB-MCL\anaconda3\envs\datamining\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
18 fits failed out of a total of 72.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mcl.NB-MCL\anaconda3\envs\datamining\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mcl.NB-MCL\anaconda3\envs\datamining\lib\site-packages\sklearn\neighbors\_classification.py", line 198, in fit
    return self._fit(X, y)
  File "C:\Users\mcl.NB-MCL\anaconda3\envs\datamining\lib\site-packages\sklearn\neighbors\_base.py", line 437, in 

KNeighborsClassifier(metric='cosine', n_neighbors=25)
k= {'algorithm': 'auto', 'metric': 'cosine', 'n_neighbors': 25} acc: 0.813


KNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Create train/test split
df_data_train, df_data_test, df_target_train, df_target_test = train_test_split(
    df_data_stemmed, df_target, test_size=0.2, random_state=42)

#dt = DecisionTreeClassifier()

for n_neighbors in range(20,25):#[1,2,3,4,5]:#1,2,3,4,5
    knn_estimator = KNeighborsClassifier(n_neighbors,metric='cosine')
    knn_estimator.fit(df_data_train, df_target_train)
    df_prediction = knn_estimator.predict(df_data_test)
    print("k= {} acc: {}".format(n_neighbors, accuracy_score(df_target_test, df_prediction)))


k= 20 acc: 0.788
k= 21 acc: 0.7928333333333333
k= 22 acc: 0.7878333333333334
k= 23 acc: 0.7898333333333334
k= 24 acc: 0.783


In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()
    
cnf_matrix = confusion_matrix(df_target_test, df_prediction)
np.set_printoptions(precision=2)
plot_confusion_matrix(cnf_matrix, classes=label_encoder.classes_)